<a href="https://colab.research.google.com/github/adakar2000/UczenieMaszynowe/blob/UseCase3/UseCase3_Adam_Karda%C5%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=6a3339bb567b6a19d3a8415f95682001c5f62fbc60bce2981780dae577baf358
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
!pip install yfinance

In [3]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [8]:
import datetime as dt
import yfinance as yf

MSFT = yf.download('MSFT', start='2024-01-01', end='2024-05-06')
MSFT.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-02,373.859985,375.899994,366.769989,370.869995,369.518921,25258600
2024-01-03,369.010010,373.260010,368.510010,370.600006,369.249908,23083500
2024-01-04,370.670013,373.100006,367.170013,367.940002,366.599579,20901500
2024-01-05,368.970001,372.059998,366.500000,367.750000,366.410309,20987000
2024-01-08,369.299988,375.200012,369.010010,374.690002,373.325012,23134000


In [9]:
class SmaCross_withSL(Strategy):
    n1 = 10
    n2 = 25

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
          price = self.data.Close
          long_sl = price * 0.99
          long_tp = price * 1.05
          if self.position.is_short:
            self.position.close()
            self.buy(size=0.5, sl=long_sl, tp=long_tp)
          elif self.position.is_long:
            pass
          else:
            self.buy(size=0.5, sl=long_sl, tp=long_tp)

        elif crossover(self.sma2, self.sma1):
          price = self.data.Close
          short_sl = price * 1.01
          short_tp = price * 0.95
          if(self.position.is_long):
            self.position.close()
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)
          elif self.position.is_short:
            pass
          else:
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)

In [10]:
bt = Backtest(MSFT, SmaCross_withSL,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                    33.72093
Equity Final [$]                  9750.012919
Equity Peak [$]                       10000.0
Return [%]                          -2.499871
Buy & Hold Return [%]                9.650284
Return (Ann.) [%]                   -7.149841
Volatility (Ann.) [%]                 3.94455
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.642838
Avg. Drawdown [%]                   -2.642838
Max. Drawdown Duration       66 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                                    5
Win Rate [%]                              0.0
Best Trade [%]                      -0.554803
Worst Trade [%]                     -1.819001
Avg. Trade [%]                    

In [11]:
opt = bt.optimize(n1=[5,7,9,11,13,15], n2=[10,15, 20,25,30,35,40], maximize='Sharpe Ratio',
                  constraint=lambda p: p.n1 < p.n2)
print(opt)

print(opt._strategy)

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2024-01-02 00:00:00
End                       2024-05-03 00:00:00
Duration                    122 days 00:00:00
Exposure Time [%]                    17.44186
Equity Final [$]                 10429.679225
Equity Peak [$]                  10429.679225
Return [%]                           4.296792
Buy & Hold Return [%]                9.650284
Return (Ann.) [%]                   13.119672
Volatility (Ann.) [%]                4.871119
Sharpe Ratio                         2.693359
Sortino Ratio                        7.326776
Calmar Ratio                        15.663373
Max. Drawdown [%]                   -0.837602
Avg. Drawdown [%]                   -0.393575
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                       3.993474
Worst Trade [%]                      0.688871
Avg. Trade [%]                    

Kluczowe Wnioski:
1. Ogólna Wydajność:
  - Całkowity Zwrot (Return): Strategia wygenerowała zwrot w wysokości 4.296792%, co jest poniżej zwrotu strategii "Buy & Hold" (9.650284%).
  - Finalny Kapitał (Equity Final): Na koniec okresu testowego, strategia zakończyła z kapitałem 10429.679225 $.
2. Ryzyko i Zmienność:
  - Volatility : Roczna zmienność strategii wynosi 4.871119%, co jest umiarkowanym poziomem zmienności.
  - Max. Drawdown : Maksymalne obsunięcie kapitału wyniosło -0.837602%, co jest bardzo niskim poziomem obsunięcia, wskazując na stosunkowo niskie ryzyko.
  - Max. Drawdown Duration: Maksymalny czas trwania obsunięcia kapitału wyniósł 6 dni, co wskazuje na bardzo krótkie okresy strat.
3. Zysk i Strata:
  - Sharpe Ratio: Wartość Sharpe Ratio wyniosła 2.693359, co jest bardzo dobrym wynikiem, sugerującym, że strategia generowała stosunkowo dobre zwroty w stosunku do podejmowanego ryzyka.
  - Sortino Ratio: Wartość Sortino Ratio wyniosła 7.326776, co wskazuje na bardzo dobrą efektywność w minimalizowaniu negatywnej zmienności.
  - Profit Factor: Wartość Profit Factor jest NaN, co sugeruje, że nie można było obliczyć tego wskaźnika, prawdopodobnie ze względu na zbyt małą liczbę transakcji.
4. Skuteczność Transakcji:
  - Win Rate: Strategia miała wskaźnik wygranych transakcji na poziomie 100%, co oznacza, że wszystkie transakcje były zyskowne.
  - Average Trade Duration: Średni czas trwania transakcji wynosił 7 dni, co może wskazywać na krótkoterminowy charakter strategii.
5. Najlepsza i Najgorsza Transakcja:
  - Best Trade: Najlepsza transakcja przyniosła 3.993474% zysku.
  - Worst Trade: Najgorsza transakcja przyniosła 0.688371% zysku, co oznacza, że wszystkie transakcje były zyskowne.

Wnioski Końcowe:
Strategia SmaCross_withSL była bardzo skuteczna pod względem wskaźnika wygranych transakcji, ponieważ wszystkie transakcje były zyskowne. Jednak generowany zwrot był niższy niż strategia "Buy & Hold".
Wskaźniki Sharpe Ratio i Sortino Ratio są bardzo wysokie, co wskazuje na wysoką efektywność strategii w stosunku do podejmowanego ryzyka.
Bardzo niski poziom obsunięcia kapitału i krótki czas trwania obsunięć wskazują na niskie ryzyko strategii.
Mała liczba transakcji (tylko 3) może sugerować, że strategia nie miała wystarczającej ilości sygnałów do podejmowania decyzji handlowych w analizowanym okresie.